In [138]:
import easyaccess as ea
from __future__ import division
from ccdBounds import *
from pyOrbfit.Orbit import Orbit
import gzip
import glob # Lists files into a directory
import tarfile
import json
import pandas as pd
import pylab
import ephem
import os
import time
import numpy as np
import requests
import sys
import easyaccess as ea
from subprocess import call
from pandas import *
from pylab import *
from astropy.io import fits
from astropy.wcs import WCS

In [139]:
def makeObsReg(regfile, ra, dec):
    with open(regfile,'w') as fout:
        fout.write('fk5; circle '+str(ra)+' '+str(dec)+' 6" #dash=1')

In [140]:
def makeSearchReg(regfile, ra, dec, PA, a, b):
    with open(regfile, 'w') as fout:
        if a>2:
            fout.write('fk5; ellipse '+str(ra)+' '+str(dec)+' '+str(a)+'" '+str(b)+'" '+str(PA-90)+' # dash=1')
        else:
            fout.write('fk5; box '+str(ra)+' '+str(dec)+' '+str(6)+'" '+str(6)+'" 0 #color=red dash=1')

In [141]:
def fit_orbit(df_obs):
    df_obs = df_obs.ix[['#' not in row['DATE_OBS'] for ind, row in df_obs.iterrows()]]   # filter comment lines
    nobs = len(df_obs)
    ralist = [ephem.hours(r) for r in df_obs['RA'].values]
    declist = [ephem.degrees(r) for r in df_obs['DEC'].values]
    datelist = [ephem.date(d) for d in df_obs['DATE_OBS'].values]
    obscode = np.ones(nobs, dtype=int)*807
    orbit = Orbit(dates=datelist, ra=ralist, dec=declist, obscode=obscode, err=0.15)
    return orbit

In [142]:
def compute_chip(rockra, rockdec, expra, expdec):
    '''
    Given the ra and dec of a point and of the center
    of an exposure, find the CCD containing that point.
    
    Returns a pair of the CCD name and number.
    '''
    deltara = 180/np.pi*ephem.degrees(rockra-expra).znorm  # compute difference in degrees (normalized between -180, +180)
    deltadec = 180/np.pi*ephem.degrees(rockdec-expdec).znorm  # the 180/pi is because ephem.Angle objects are natively in radians
    ccdname = 'None'
    for k in ccdBounds:
        if deltara > ccdBounds[k][0] and deltara < ccdBounds[k][1] and deltadec > ccdBounds[k][2] and deltadec < ccdBounds[k][3]:
            ccdname = k
    return ccdname, ccdNum[ccdname]

In [143]:
def populateImgs(obs_properties, canid,directory,raw_fname,flist, ob):
    
    db = ea.connect(section='desoper')
    cursor=db.cursor()
    
    exposures = read_csv('exposures.csv') 
    exp_values = exposures.ix[:,['expnum','t_eff']]
    
    os.mkdir(directory+'PermFiles/NotTempImgs_'+raw_fname) #makes the directory for all images needed for website.
    
    #search_array=pandas.DataFrame.from_items([('expnum',[]),('refccd',[]),('refpng',[]), ('refdate',[]),('reftef',[]),('compimages',[]),('compexp',[]),('compccd',[]),('compdate',[]),('comptef',[])])
    
    #def getkey(item):
     #       return item[0]
    sid=1    
    for i in obs_properties.index:
        #print "entering row"+str(i), time.ctime()
        
        commandstr='ds9 '
        
        refnum=obs_properties['expnum'][i]
        refccd=obs_properties['ccd'][i]
        #refccd=obs_properties['ccdnum'][i]
        refband=obs_properties['band'][i]
        refdate=str(ephem.date(obs_properties['date'][i]))
        ref_ra=ephem.hours(obs_properties['can_ra'][i])
        ref_dec=ephem.degrees(obs_properties['can_dec'][i])
        ref_objid=obs_properties['objid'][i]
        ref_PA=obs_properties['PA'][i]
        ref_a=obs_properties['a'][i]
        ref_b=obs_properties['b'][i]
        #CompImgs=[]
        expnums=[]
        refpng=''
        reftef=round(float(exp_values[exp_values['expnum']==refnum]['t_eff']),3)
        #refisinobs=obs_properties['isinobs'][i]
        
        
        breaktime=False
        quicklabel=[]
        
        for thumb in flist:
            #print r"     checking thumb"+thumb, time.ctime()
            os.chdir(directory+raw_fname+'/'+thumb)
            fit_list = glob.glob('*fits')
        
            for f in fit_list:
                h=fits.open(f)
                if h[0].header["EXPNUM"]==refnum and h[0].header['CCDNUM']==refccd:
                    #print r"         Found correct thumb", time.ctime()
                    reffit=f
                    refpng='PermFiles/NotTempImgs_'+raw_fname+'/'+reffit[:-5]+'Circ.png' 
                    if ob:
                        makeObsReg('temp.reg',ref_ra,ref_dec)
                        cmd="INSERT INTO scheibne.Images (canid, objid,path,ref,t_eff, ccd, expnum, Date_obs) \
                            VALUES ('"+canid+"', '"+str(ref_objid)+"','"+refpng+"' ,'Yes',"+str(reftef)+","+str(refccd)+","+str(refnum)+",'"+refdate+"')"
                        cursor.execute(cmd)
                    else:
                        makeSearchReg('temp.reg',ref_ra, ref_dec, ref_PA, ref_a, ref_b)
                        cmd="INSERT INTO scheibne.SearchImgs (canid, searchid,path,ref,t_eff, ccd, expnum, Date_obs) \
                            VALUES ('"+canid+"', '"+str(sid)+"','"+refpng+"' ,'Yes',"+str(reftef)+","+str(refccd)+","+str(refnum)+",'"+refdate+"')"
                        cursor.execute(cmd)
                    commandstr+=f+' -scale zscale '
                    
                    for g in fit_list:
                        j=fits.open(g)
                        jnum=j[0].header['EXPNUM']
                        if (j[0].header['BAND']==refband and jnum != refnum and not (jnum in expnums)):
                            try:
                                teff=float(exp_values[exp_values['expnum']==jnum]['t_eff'])
                            except:
                                teff=0
                                #print jnum
                            pnglabel='PermFiles/NotTempImgs_'+raw_fname+'/'+g[:-5]+'Circ.png'
                            quicklabel+=[g[:-5]+'Circ.png']
                            date=str(j[0].header['DATE-OBS'])
                            nicedate=date[0:4]+'/'+date[5:7]+'/'+date[8:10]+' '+date[11:19]
                            compccd=j[0].header['CCDNUM']
                            #CompImgs.append([teff,pnglabel, jnum, j[0].header['CCDNUM'], nicedate])
                            expnums+=[jnum]
                            if ob:
                                cmd="INSERT INTO scheibne.Images (canid, objid,path,ref,t_eff, ccd, expnum, Date_obs) \
                                VALUES ('"+canid+"', '"+str(ref_objid)+"','"+pnglabel+"' ,'No',"+str(teff)+","+str(compccd)+","+str(jnum)+",'"+nicedate+"')"
                                cursor.execute(cmd)
                            else:
                                cmd="INSERT INTO scheibne.searchimgs (canid, searchid,path,ref,t_eff, ccd, expnum, Date_obs) \
                                VALUES ('"+canid+"', '"+str(sid)+"','"+pnglabel+"' ,'No',"+str(teff)+","+str(compccd)+","+str(jnum)+",'"+nicedate+"')"
                                cursor.execute(cmd)
                            commandstr+=' -file '+g
                        j.close()
                    commandstr+=' -single -region load all temp.reg -height 288 -width 288 -colorbar no -bg black '
                    for q in range(1,len(quicklabel)+1):
                        commandstr+=' -zoom to fit -saveimage png '+quicklabel[-q]+' -frame prev '
                    commandstr+='-zoom to fit -saveimage png '+reffit[:-5]+'Circ.png'+' -exit'
                    os.system(commandstr)
                    os.system('cp '+directory+raw_fname+'/'+thumb +'/*.png '+directory+'PermFiles/'+'NotTempImgs_'+raw_fname)
                    flist.remove(thumb)
                    breaktime=True
                    #print r"         Done writing row", time.ctime()
                    break 
                h.close()
            if breaktime:
                break
        
        sid+=1
        #sortedcomps=sorted(CompImgs, key=getkey, reverse=True)
        
        #tempframe=pandas.DataFrame.from_items([('expnum', refnum),('refccd',refccd), ('refpng',[ refpng]), ('refdate', [refdate]),('reftef',[reftef]),('compimages', [[x[1] for x in sortedcomps]]),('compexp',[[x[2] for x in sortedcomps]]), ('compccd',[[x[3] for x in sortedcomps]]),('compdate',[[x[4] for x in sortedcomps]]),('comptef',[[round(x[0],3) for x in sortedcomps]])])                
        #search_array=search_array.append(tempframe, ignore_index=True)
    os.chdir(directory)
    db.close()
    #return search_array

In [144]:
def findExposures(canid):
    all_exps = read_csv('exposures.csv')
    df = getCanFromDatabase(canid)
    orb = fit_orbit(df)
    matches=DataFrame() #Dataframe is a panda function.  Creates an empty dataframe
    
    #print df
    
    #Checks to see the position of the candidate in the orbit and does that position lie in any ccd
    for i in all_exps[all_exps['t_eff']>.3].index:
        e_ra, e_dec=all_exps['ra'][i],all_exps['dec'][i]
        pos=orb.predict_pos(all_exps['date'][i]) #Predicts position 
        pos_ra, pos_dec=pos['ra'], pos['dec']
        ccdname, ccdnum = compute_chip(pos_ra, pos_dec, e_ra, e_dec)
        if ccdnum>0:
            temp=all_exps.loc[i,['expnum','date','nite','band','t_eff']]
            temp['ccd']=ccdnum
            temp['PA']=pos['err']['PA']
            temp['a']=pos['err']['a'] #Semimajor axis of error elipse
            temp['b']=pos['err']['b'] #Semiminor axis of error elipse
            #temp['can_ra']=pos_ra
            #temp['can_dec']=pos_dec
            #temp['isinobs']=all_exps['expnum'][i] in list(df['expnum']) #True if it should be in top section of array aka we know if should be in that exposure
            if all_exps['expnum'][i] in list(df['EXPNUM']):
                temp['isinobs']=True
                temp['objid']=int(df[df['EXPNUM']==all_exps['expnum'][i]]['OBJID'])
                temp['mag']=float(df[df['EXPNUM']==all_exps['expnum'][i]]['MAG'])
                temp['ml_score']=float(df[df['EXPNUM']==all_exps['expnum'][i]]['ML_SCORE'])
                #temp['exptime']=float(df[df['EXPNUM']==all_exps['expnum'][i]]['EXPTIME'])
                temp['can_ra']=float(ephem.hours(df[df['EXPNUM']==all_exps['expnum'][i]]['RA'].get_values()[0]))
                temp['can_dec']=float(ephem.degrees(df[df['EXPNUM']==all_exps['expnum'][i]]['DEC'].get_values()[0]))
            else:
                temp['isinobs']=False
                temp['objid']=-99
                temp['mag']=-99
                temp['ml_score']=-99
                #temp['exptime']=-99
                temp['can_ra']=pos_ra
                temp['can_dec']=pos_dec
            matches=matches.append(temp,ignore_index=True)
    return matches

In [145]:
def sendRequest(frame,username,password):
    ra=list(np.degrees(frame['can_ra']))
    dec=list(np.degrees(frame['can_dec']))
    bands='[g,r,i,z]'
    req='http://descut.cosmology.illinois.edu:8000/api?username='+username+'&password='+password+'&ra=%s&dec=%s&bands=%s' % (ra,dec,bands)
    submit = requests.get(req)
    return submit.json()['job'] # Returns job ID, long string

In [146]:
def open_json(jsonName):
    f = open(jsonName, "r")
    s=f.read()
    type(s)
    book = json.loads(s)
    return book

In [147]:
def fetchFiles(jobid):
   
    while True:
        os.system('wget -O file_list_'+jobid+'.txt http://descut.cosmology.illinois.edu:8000/static/uploads/lzullo/results/'+jobid+'/file_list.txt')
        #os.system('wget -O file_list_'+jobid+'.txt http://descut.cosmology.illinois.edu:8000/static/uploads/scheibne/results/'+jobid+'/file_list.txt')
        myfile=open(os.getcwd()+'/file_list_'+jobid+'.txt')
        mylines=myfile.readlines()
        if mylines==[]:
            time.sleep(10)
        else:
            break
            
            
    newlines=[]
    for l in mylines:
        if l[-5:-1]=='fits' or l[-5:-1]=='json':
            newlines+=['http://descut.cosmology.illinois.edu:8000'+l[37:]]
    newfilelist=open('newfile_list.txt','w')
    newfilelist.writelines(newlines)
    newfilelist.close()
    myfile.close()
    try:
        os.mkdir('temp_'+jobid) #will this work if directory already exists? Yes. Maybe just use the job id as the directory name
    except:
        print "temp_"+jobid+" already exists"
    os.chdir('temp_'+jobid)
    os.system('wget -i '+'../newfile_list.txt')
    os.chdir('..')
    
    #os.remove('file_list_'+jobid+'.txt')
    return 'temp_'+jobid

In [148]:
#takes a directory, preferablly the directory given from fetchFiles. Creates sub directories called thumbs_0, thumbs_1 ,...
#populates each thumb directory with fits files sharing the same ra/dec center. 
def organizeTempDirectory(tempDirectory):
    os.chdir(tempDirectory)
    jsonlist=glob.glob('*.json*')
    for i in range(len(jsonlist)):
        try:
            os.mkdir('thumbs_'+str(i))
        except:
            print 'thumbs_'+str(i)+' already exists'
        jarray=open_json(jsonlist[i])
        for j in jarray:
            try: 
                os.system('mv '+j['png_name'][:-3]+'fits'+' thumbs_'+str(i)) 
            except:
                print "Could not move file"
    os.chdir('..')

In [149]:
def getCanFromDatabase(canid):
    db = ea.connect(section='desoper')
    
    query = "select DATE_OBS, RA, DEC, EXPNUM, BAND, CCD, MAG, ML_SCORE, objects.objid from scheibne.OBJECTS inner join scheibne.linker on \
objects.objid=linker.objid where linker.canid='"+canid+"'"
    
    result = db.query_to_pandas(query)
    
    db.close()
    
    return result

In [150]:
def populateSearchTable(canid, expected):
    db = ea.connect(section='desoper')
    cursor=db.cursor()
    delcom="delete from scheibne.SearchExps where canid='"+canid+"'"
    cursor.execute(delcom)
    sid=1
    cmds=[]
    for i in expected.index:
        cmds += ["INSERT INTO scheibne.SearchExps (CANID,SearchID,RA, DEC, expnum, Band, CCD, PA, a, b, Date_obs) \
        VALUES ('"+str(canid)+"',"+str(sid)+",'"+str(ephem.hours(expected['can_ra'][i]))+"','"+str(ephem.degrees(expected['can_dec'][i]))+"',"+str(expected['expnum'][i])+",'"+str(expected['band'][i])+"',"+str(expected['ccd'][i])+","+str(expected['PA'][i])+","+str(expected['a'][i])+","+str(expected['b'][i])+", '"+str(ephem.date(expected['date'][i]))+"')"]
        sid+=1
    for c in cmds:
        cursor.execute(c)
    db.close()



In [161]:
def cleanImgs(canid, directory):
    db = ea.connect(section='desoper')
    cursor=db.cursor()
    
    os.chdir(directory+"PermFiles")
    
    #ssh username@domain.com 'rm /some/where/some_file.war'
    
    query = "select path from scheibne.Images where (canid='"+canid+"') and rownum=1"
    pandaPath=db.query_to_pandas(query)
    if len(pandaPath['PATH'])>0: 
        Path=str(pandaPath['PATH'][0])[10:-41] #contingent of form of file path
        os.system("rm -r "+Path)
        os.system('ssh scheibne@dessne.cosmology.illinois.edu rm -r /home2/SNWG/web/display/new/PermFiles/' + Path)
    
    delcom="delete from scheibne.Images where canid='"+canid+"'"
    cursor.execute(delcom)
    
    query = "select path from scheibne.searchImgs where (canid='"+canid+"') and rownum=1"
    pandaPath=db.query_to_pandas(query)
    if len(pandaPath['PATH'])>0: 
        Path=str(pandaPath['PATH'][0])[10:-41] #contingent of form of file path
        os.system("rm -r "+Path)
        os.system('ssh scheibne@dessne.cosmology.illinois.edu rm -r /home2/SNWG/web/display/new/PermFiles/' + Path)
    
    
    
    delcom="delete from scheibne.SearchImgs where canid='"+canid+"'"
    cursor.execute(delcom)
    
    db.close()
    os.chdir(directory)
    

In [162]:
def writeOrb(canid):
    df = getCanFromDatabase(canid)
    orb = fit_orbit(df)
    db = ea.connect(section='desoper')
    cursor=db.cursor()
    elements, err = orb.get_elements()
    
    delcom="delete from scheibne.Orbit where canid='"+canid+"'"
    cursor.execute(delcom)
    if elements['a'] > 0:
        a=str(round(elements['a'],2))
    else:
        a=str(-99)
    if elements['aop']>0:
        aop=str(round(elements['aop'],2))
    else:
        aop=str(-99)
    if elements['aop']>0:
        e=str(round(elements['e'],2))
    else:
        e=str(-99)
    if elements['i']>0:
        i=str(round(elements['i'],2))
    else:
        i=sr(-99)
    if elements['lan']>0:
        lan=str(round(elements['lan'],2))
    else:
        lan=str(-99)
    if elements['top']>0:
        top=str(ephem.date(elements['top']))
    else:
        top="err"
    if err['a']>0:
        err_a=str(round(err['a'],6))
    else:
        err_a=str(-99)
    if err['aop']>0:
        err_aop=str(round(err['aop'],6))
    else:
        err_aop=str(-99)
    if err['e']>0:
        err_e=str(round(err['e'],6))
    else:
        err_e=str(-99)
    if err['i']>0:
        err_i=str(round(err['i'],6))
    else:
        err_i=str(-99)
    if err['lan']>0:
        err_lan=str(str(round(err['lan'],6)))
    else:
        err_lan=str(-99)
    if err['top']>0:
        err_top=str(round(err['top'],6))
    else:
        err_top=str(-99)
    c = "INSERT INTO scheibne.Orbit (CANID, a ,aop, e, i, lan, top, err_a, err_aop, err_e, err_i, err_lan, err_top) \
    VALUES ('"+canid+"',"+a+","+aop+","+e+","+i+","+lan+",'"+top+"',"+err_a+", "+err_aop+", "+err_e+", "+err_i+", "+err_lan+", "+err_top+")"
  
    cursor.execute(c)
    
    db.close()

In [163]:
def main(directory, canid):
    os.chdir(directory)
    
    matches = findExposures(canid)
    
    writeOrb(canid)
    
    observed = matches[matches['isinobs']==True]
    expected = matches[matches['isinobs']==False]
    
    username = raw_input('Enter username ')
    password = raw_input('Enter password ')
    
    rawFileObs=sendRequest(observed, username, password)
    rawFileSearch=sendRequest(expected, username, password)
    
    raw_fnameObs=fetchFiles(rawFileObs)
    raw_fnameSearch=fetchFiles(rawFileSearch)
    
    organizeTempDirectory(raw_fnameObs)
    organizeTempDirectory(raw_fnameSearch)
    
    populateSearchTable(canid, expected)
    
    os.chdir(directory+'/'+raw_fnameObs)
    flistObs = glob.glob('thumbs*')
    os.chdir(directory+'/'+raw_fnameSearch)
    flistSearch = glob.glob('thumbs*')
    
    os.chdir(directory)
    
    cleanImgs(canid, directory)
    
    populateImgs(observed,canid,directory, raw_fnameObs, flistObs, True) #Need a directory called PermFiles
    populateImgs(expected,canid,directory, raw_fnameSearch, flistSearch, False)
    
    os.system('rm -r '+raw_fnameObs)
    os.system('rm -r '+raw_fnameSearch)
    copyFiles(raw_fnameObs)
    print 'done'#, time.ctime()
#print pag

In [164]:
def copyFiles(raw_fnameObs):
    os.system('scp -r /Users/lynuszullo/pyOrbfit/PermFiles scheibne@dessne.cosmology.illinois.edu:/home2/SNWG/web/display/new')

In [167]:
if __name__ == '__main__':
    start_time = time.time()
    main('/Users/lynuszullo/pyOrbfit/','cand002')
    #main('/Users/ColinS/Documents/TNOSearch/','cand003')
    minutes = (((time.time() - start_time)) / 60)
    
    print("%s minutes" % (minutes))

Connecting to DB ** desoper ** ...
Connecting to DB ** desoper ** ...
Connecting to DB ** desoper ** ...
Enter username lzullo
Enter password lzu70chips
Connecting to DB ** desoper ** ...
Connecting to DB ** desoper ** ...
Connecting to DB ** desoper ** ...
Connecting to DB ** desoper ** ...
done
1.87204318444 minutes
